In [31]:
# Condensing (Smoothing Prep)

In [32]:
# Imports

import pandas as pd
import numpy as np

In [33]:
p_path = 'p9'
file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/'+p_path+'_master.csv'
data = pd.read_csv(file)

In [34]:
# move timestamp, session_num to first two columns
cols = np.array(data.columns).tolist()
cols.remove('timestamp')
cols.remove('session_num')
cols.remove('engagement')
cols = ['session_num', 'timestamp', 'engagement'] + cols
data = data[cols]

In [35]:
data = data.sort_values(['session_num', 'timestamp'], ascending=[True, True])

In [36]:
orig_len = len(data)

In [37]:
data.head()

,session_num,timestamp,engagement,of_confidence,of_success,of_gaze_0_x,of_gaze_0_y,of_gaze_0_z,of_gaze_1_x,of_gaze_1_y,...,p_skill_OS,p_skill_EM,p_no_game,a_harmonicity,a_intensity,a_mfcc_0,a_mfcc_1,a_pitch_frequency,a_pitch_strength,participant
0,3.1,64.4333,1,0.03,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,-200.0,51.398,756.292,309.607,878.153,0.514,9
1,3.1,64.4667,1,0.03,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,-200.0,51.398,756.292,309.607,878.153,0.514,9
2,3.1,64.5000,1,0.03,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,-200.0,51.507,1053.846,283.242,6470.416,0.696,9
3,3.1,64.5333,1,0.38,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,-200.0,51.507,1053.846,283.242,6470.416,0.696,9
4,3.1,64.5667,1,0.03,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,-200.0,51.507,1053.846,283.242,6470.416,0.696,9


In [38]:
data.columns

Index(['session_num', 'timestamp', 'engagement', 'of_confidence', 'of_success',
       'of_gaze_0_x', 'of_gaze_0_y', 'of_gaze_0_z', 'of_gaze_1_x',
       'of_gaze_1_y', 'of_gaze_1_z', 'of_gaze_angle_x', 'of_gaze_angle_y',
       'of_gaze_distance', 'of_pose_Tx', 'of_pose_Ty', 'of_pose_Tz',
       'of_pose_Rx', 'of_pose_Ry', 'of_pose_Rz', 'of_pose_distance',
       'of_AU01_c', 'of_AU02_c', 'of_AU04_c', 'of_AU05_c', 'of_AU06_c',
       'of_AU07_c', 'of_AU09_c', 'of_AU10_c', 'of_AU12_c', 'of_AU14_c',
       'of_AU15_c', 'of_AU17_c', 'of_AU20_c', 'of_AU23_c', 'of_AU25_c',
       'of_AU26_c', 'of_AU28_c', 'of_AU45_c', 'op_num_people', 'p_games_total',
       'p_games_session', 'p_mistakes_total', 'p_mistakes_session',
       'p_mistakes_game', 'p_ts_robot_talked', 'p_ts_game_start',
       'p_ts_attempt', 'p_aptitude', 'p_diff_1', 'p_diff_2', 'p_diff_3',
       'p_diff_4', 'p_diff_5', 'p_skill_NC', 'p_skill_OS', 'p_skill_EM',
       'p_no_game', 'a_harmonicity', 'a_intensity', 'a_mfcc_0', 

# 1. Remove Beginning of Videos/Sessions

In [39]:
len(data.loc[data['engagement']==1])/len(data)

0.4748476685963788

In [40]:
# Remove data @ beginning of each session before first game has been played 

last_session = -1
session_start = -1 
track = False

# tuples: (session_start, first_game_start)
windows_to_remove = [] 

for i,r in data.iterrows():
    # Check if new session started
    if last_session != r['session_num']:
        session_start = i
        last_session = r['session_num']
        track = True 
    
    if track:
        # check if we've found our first game start! 
        if r['p_no_game'] == 0: 
            windows_to_remove.append((session_start, i))
            track = False 

In [41]:
for i in windows_to_remove:
    print(i)

(0, 0)
(26808, 27738)
(162787, 162787)
(242321, 242981)
(259124, 259424)


In [42]:
# Remove data at beginning
# Iterate backwards! 

n = len(windows_to_remove)
for i in range(n-1, -1, -1):
    rem = windows_to_remove[i]

    if (rem[0] == rem[1]):
        continue

    seg1 = data.loc[:rem[0]-1]
    seg2 = data.loc[rem[1]:, :]
    data = seg1.append(seg2, sort=True)
    
data = data.reset_index(drop=True)

In [43]:
curr_len = len(data)
print((orig_len-curr_len)/orig_len)

0.005132020549805038


# 2. TS_game_played
=> Help Indicate "True" Ending of Videos/Sessions

In [44]:
# Add "Time Since Game Played" Column
# 0 if game in progress, otherwise 

last_session = -1
last_success = -1

data['ts_game_played'] = 0

for i,r in data.iterrows():
    # Check if new session started
    if last_session != r['session_num']:
        last_session = r['session_num']
        last_success = -1
    
    # 1 -- game played at this row, do nothing
    if r['p_no_game']==0:
        last_success = i
    # 2 -- game not played, no game this session
    elif last_success == -1: # should be REMOVED in earlier code! 
        data.loc[i, 'ts_game_played'] = -1
    # 3 -- game not played, game this session
    else:
        data.loc[i, 'ts_game_played'] = (i-last_success)*(0.1/3)

In [45]:
# save pre-smoothing data

file = '../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master/'+p_path+'_master.csv'
data.to_csv(file, index=False)